# Explore signal/price movement relationship

This is an example notebook to explore whether a trading signal results to profitable trades.

- It explores a small set of pairs on Uni v3 on Polygon
- Both long and short are considered (though shorts might be theoretical only, if such a lending market doest not exist in the point of time)

In [1]:
import datetime

import pandas as pd

from tradingstrategy.client import Client
from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.execution_context import notebook_execution_context
from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.strategy_module import read_strategy_module
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse, load_trading_and_lending_data
from tradeexecutor.strategy.universe_model import UniverseOptions


def create_trading_universe(
    ts: datetime.datetime,
    client: Client,
    execution_context,
    universe_options,
) -> TradingStrategyUniverse:
    # We limit ourselves to price feeds on Uniswap v3 and Quickswap on Polygon,
    # as there are multiple small or dead DEXes on Polygon
    # which also have price feeds but not interesting liquidity
    dataset = load_trading_and_lending_data(
        client,
        execution_context=execution_context,
        universe_options=universe_options,
        # Ask for all Polygon data
        chain_id=ChainId.polygon,
        exchange_slugs={"uniswap-v3"},
        reserve_asset_symbols={"USDC"},
        asset_symbols={"LINK", "WMATIC", "WETH", "BAL"},
        trading_fee=0.0005,
        time_bucket=TimeBucket.d1,
        stop_loss_time_bucket=TimeBucket.h1,
    )

    # Filter down the dataset to the pairs we specified
    universe = TradingStrategyUniverse.create_from_dataset(dataset)

    return universe


client = Client.create_jupyter_client()

strategy_universe = create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    notebook_execution_context,
    UniverseOptions(start_at=pd.Timestamp("2022-01-01"), end_at=pd.Timestamp("2023-11-01")),
)

data_universe = strategy_universe.data_universe

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


TypeError: UniverseOptions.__init__() got an unexpected keyword argument 'start'

## Github mode

Set chart output mode to static images that Github website notebook viewer can display.

In [ ]:
from tradeexecutor.utils.notebook import OutputMode, setup_charting_and_output


setup_charting_and_output(OutputMode.static, image_format="png")

## Explore signal vs price change

Create a function `calculate_signal_vs_profit` which calculates 
- Signal (naive momentum)
- Profit: the last and the best future price we can get
- Allows us to play around with different time windows
- Split between shorts and longs


In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.frequencies import to_offset

from tradingstrategy.utils.forward_fill import forward_fill
from tradingstrategy.chain import ChainId
from tradingstrategy.pair import DEXPair

def calculate_signal_vs_profit(df, signal_window: pd.Timedelta, profit_window: pd.Timedelta) -> pd.DataFrame:
    """Calculate signals and profits for all incoming candles."""
    
    # Create entries for past price to be used for signal
    # and future price (used for the price correlation)
    signal_offset = to_offset(signal_window)
    profit_offset = to_offset(profit_window)

    df["prev"] = df["close"].shift(freq=signal_offset)
    df["next"] = df["open"].shift(freq=-profit_offset)

    # Calculate signal from the past and price difference to the future
    df["signal"] = (df["prev"] - df["open"]) / df["open"]
    df["price_diff"] = (df["next"] - df["open"]) / df["open"]  # Get the profit on the final day of profit window

    # On negative signals, we go short.
    # On zero signal and lack of data set side to NA
    df["side"] = pd.NA
    df.loc[df["signal"] > 0, "side"] = "long"
    df.loc[df["signal"] < 0, "side"] = "short"

    # Max and min price wihtin the profit window will determine the profit for longs and shorts respective
    df["max_future_price"] = df["close"].rolling(profit_window.days).max().shift(-profit_window.days) # Get the max profit on the profit window, assuming take profit %
    df["min_future_price"] = df["close"].rolling(profit_window.days).min().shift(-profit_window.days) # Get the max profit on the profit window, assuming take profit %    
    
    # Calculate profit separately for longs and shorts
    # using Pandas Mask
    # https://stackoverflow.com/a/33770421/315168
    #
    # We calculate both profit after X time,
    # and also max take profit, assuming
    # we could do a perfect trailing stop loss
    #
    longs = (df["side"] == "long")
    shorts = (df["side"] == "short")
    df.loc[longs, "profit"] = df["price_diff"]
    df.loc[shorts, "profit"] = -df["price_diff"]
    df.loc[longs, "profit_max"] = (df["max_future_price"] - df["open"]) / df["open"]  # Get the profit based on max price
    df.loc[shorts, "profit_max"] = -(df["min_future_price"] - df["open"]) / df["open"]  # Get the profit based on max price

    df.loc[longs, "desc"] = df.agg('{0[pair]} long'.format, axis=1)
    df.loc[shorts, "desc"] = df.agg('{0[pair]} short'.format, axis=1)

    
    return df
     


Create a function `calculate_signal_vs_price_for_pair` which calculates 
- Calculates the signal vs. for certain trading pair
- Bundle few different pairs to the same `DataFrame` so we can examine them together


In [ ]:
def calculate_signal_vs_price_for_pair(pair: DEXPair) -> pd.DataFrame:
    """Calculate signal vs. profit ratio for an individual pair."""
    df = data_universe.candles.get_candles_by_pair(pair)
    assert df is not None

    df = df.copy()

    # Make sure there are no gaps in the data
    df = forward_fill(
        df, 
        freq=data_universe.time_bucket.to_frequency(), 
        columns=("open", "high", "low", "close")
    )

    df["pair"] = pair.get_ticker()
    df = calculate_signal_vs_profit(
        df,
        #signal_window=pd.DateOffset(days=7),
        #profit_window=pd.DateOffset(days=7),
        signal_window=pd.Timedelta(days=8),
        profit_window=pd.Timedelta(days=8),
    )

    
    return df

pairs = data_universe.pairs
eth = calculate_signal_vs_price_for_pair(pairs.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "WETH", "USDC")))
link = calculate_signal_vs_price_for_pair(pairs.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "LINK", "USDC")))
matic = calculate_signal_vs_price_for_pair(pairs.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "WMATIC", "USDC")))

df = pd.concat([eth, link, matic])

pd.set_option('display.min_rows', 36)
display(df)

## Plot signal vs. price samples

Different scatter charts to examine if there is a correlation between the signal and the profit.

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="signal", y="profit_max", trendline="ols", color="desc", title="Max take profit")
fig.show()

fig = px.scatter(df, x="signal", y="profit", trendline="ols", color="desc", title="Profit last day")
fig.show()

### Long only vs. short 

Examine trade types separately to better see if one side is clearly superior.

In [ ]:
df_long_only = df.loc[df["side"] == "long"]

fig = px.scatter(df_long_only, x="signal", y="profit_max", trendline="ols", color="desc", title="Long only max take profit")
fig.show()

fig = px.scatter(df_long_only, x="signal", y="profit", trendline="ols", color="desc", title="Long only profit last day")
fig.show()

In [ ]:
df_short_only = df.loc[df["side"] == "short"]

fig = px.scatter(df_short_only, x="signal", y="profit_max", trendline="ols", color="desc", title="Short only max take profit")
fig.show()

fig = px.scatter(df_short_only, x="signal", y="profit", trendline="ols", color="desc", title="Short only profit last day")
fig.show()

### Filtered signal

Remove low signals below a certain threshold and see if it affects the correlation between the signal and the profit.

In [ ]:

threshold = 0.05

filtered_df = df.loc[abs(df["signal"]) >= threshold]

fig = px.scatter(filtered_df, x="signal", y="profit_max", trendline="ols", color="desc", title=f"Filtered signal at {threshold} max take profit")
fig.show()

fig = px.scatter(filtered_df, x="signal", y="profit", trendline="ols", color="desc", title=f"Filtered signal at {threshold} profit last day")
fig.show()